In [24]:
import pycountry
import PyPDF2 
import textract
import re

In [25]:
# Initialize list sequence to store names of Countries
countries = [
    'Afghanistan','Albania','Algeria','Angola','Antigua and Barbuda',
    'Argentina','Armenia','Aruba','Australia','Austria','Azerbaijan',
    'The Bahamas','Bahrain','Bangladesh','Barbados','Belarus','Belgium', 
    'Belize','Benin','Bhutan','Bolivia','Bosnia and Herzegovina',
    'Botswana','Brazil','Brunei','Bulgaria','Burkina Faso','Burma',
    'Burundi','Cabo Verde','Cambodia','Cameroon','Canada','Central African Republic',
    'Chad','Chile','China','Colombia','Comoros','Congo, Democratic Republic of the',
    'Congo, Republic of the','Costa Rica',"Cote d'Ivoire",'Croatia','Cuba','Curaçao',
    'Cyprus','Czech Republic', 'Denmark','Djibouti', 'Dominican Republic','Ecuador',
    'Egypt','El Salvador','Equatorial Guinea', 'Eritrea','Estonia','Eswatini','Ethiopia',
    'Fiji', 'Finland', 'France','Gabon','The Gambia','Georgia','Germany','Ghana','Greece',
    'Guatemala','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','Hong Kong','Hungary',
    'Iceland','India','Indonesia','Iran','Iraq','Ireland','Israel','Italy','Jamaica','Japan',
    'Jordan','Kazakhstan','Kenya',"Korea, Democratic People's Republic of",'Korea, Republic of',
    'Kosovo','Kuwait','Kyrgyz Republic',"Laos",'Latvia','Lebanon','Lesotho','Liberia','Lithuania',
    'Luxembourg','Macau','Macedonia','Madagascar','Malawi','Malaysia','Maldives','Mali','Malta',
    'Marshall Islands','Mauritania','Mauritius','Mexico','Micronesia, Federated States of',
    'Moldova','Mongolia','Montenegro','Morocco','Mozambique','Namibia','Nepal','Netherlands',
    'New Zealand','Nicaragua','Niger','Nigeria','Norway','Oman','Pakistan','Palau','Panama',
    'Papua Guinea New','Paraguay','Peru','Philippines','Poland','Portugal','Qatar','Romania',
    'Russia','Rwanda','St. Lucia','St. Vincent and the Grenadines','Saudi Arabia','Senegal','Serbia',
    'Seychelles','Sierra Leone','Singapore','Slovakia','Slovenia','Solomon Islands','South Africa',
    'South Sudan','Spain','Sri Lanka','Sudan','Suriname','Sweden','Switzerland','Syria','Taiwan',
    'Tajikistan','Tanzania','Thailand','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia',
    'Turkey','Turkmenistan','Uganda','Ukraine','United Arab Emirates','United Kingdom',
    'United States of America','Uruguay','Uzbekistan','Venezuela', 'Vietnam','Zambia','Zimbabwe',
    'Libya','St. Maarten','Somalia','Yemen' #Libya, St. Marteen, Somalia and Yemen are Special Cases
]


In [26]:
len(countries)

187

In [27]:
# Function to Parse the pdf file
def parse_file(filename): 
    # Read the file.
    pdfFileObj = open(filename,'rb')
    # The pdfReader variable is the readable object that will be parsed.
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #Parse through all the pages.
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    # The while loop will read each page.
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
    # This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
        text = text
    # If the above returns as False, run the OCR library textract to #convert scanned/image based PDF files into text.
    else:
        text = textract.process(fileurl, method='tesseract', language='eng')
    return text

text = parse_file('2018-TIP-Report.pdf')
    

In [28]:
# Function to Clean the parsed pdf
def clean_text(text):
    text = text.strip().replace("\n", "").replace("˜", "fi").replace("˚", "ft").replace("Sincerely,iiiiii","")\
                .replace("Trafiick","Traffick").replace("™", "'").replace("trafiick","traffick")\
                .replace('ﬁprevalence rateﬂ','"prevalence rate"').replace("˚ight","fight")\
                .replace("/", "").replace("  REPUBLIC OF THE:", " REPUBLIC OF THE:")\
                .replace("DEMOCRATIC PEOPLE'S  REPUBLIC OF", "DEMOCRATIC PEOPLE'S REPUBLIC OF")\
                .replace("PAPUA NEW GUINEA:", "PAPUA GUINEA NEW:").replace("⁄", "").replace("\\","")\
                .replace("ST. VINCENT AND  THE GRENADINES", "ST. VINCENT AND THE GRENADINES")\
                .replace("UNITED STATES  OF AMERICA", "UNITED STATES OF AMERICA")\
                .replace("461SPECIAL CASE: LIBYA", " ").replace("462SPECIAL CASE: LIBYA", " ")\
                .replace("463SPECIAL CASE: ST. MAARTEN", " ").replace("464SPECIAL CASE: SOMALIA", " ")\
                .replace("465SPECIAL CASE: SOMALIA", " ").replace("466SPECIAL CASE: YEMEN", " ")\
                .replace("467SPECIAL CASE: YEMEN", " ").replace("468SPECIAL CASE: YEMEN", " ")
    return text 

cleaned_text = clean_text(text)


In [ ]:
# Function to Extract each country data from text
def extract_info(countries_list,text):
    # Create list to query info for each country
    countries_up = []
    for i in countries:
        if i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']:
            countries_up.append("SPECIAL CASE: "+i.upper())
        else: 
            countries_up.append(i.upper()+":")        
    new_list = []
    a = text.split(countries_list[0])
    for i in range(len(countries_list)-1):
        a = a[1].split(countries_list[i+1])
        new_list.append(a[0])
        if i+1 == len(countries_list)-1:
            new_list.append(a[1].split("469RELEVANT INTERNATIONAL CONVENTIONS")[0])
    return new_list

country_info = extract_info(countries,cleaned_text)
len(country_info)


In [31]:
# Function to Extract each country data from text
def extract_info(countries_list,text):
    # Create list to query info for each country
    countries_up = []
    for i in countries:
        if i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']:
            countries_up.append("SPECIAL CASE: "+i.upper())
        else: 
            countries_up.append(i.upper()+":")
    new_list = []
    a = text.split(countries_up[0])
    for i in range(len(countries_up)-1):
        a = a[1].split(countries_up[i+1])
        new_list.append(a[0])
        if i+1 == len(countries_up)-1:
            new_list.append(a[1].split("469RELEVANT INTERNATIONAL CONVENTIONS")[0])
    return new_list

country_info = extract_info(countries,cleaned_text)
len(country_info)


187

In [32]:
country_dict = {i.upper():[{j:k}] for i,j,k in zip(countries, ["INTRO"]*187, country_info)}
special = [s.upper() for s in ['Libya','St. Maarten','Somalia','Yemen']]
special_country_dict = {s:country_dict[s] for s in special}
print(len(country_dict.values()))
print(len(special_country_dict))

187
4


In [33]:
def removekey(d,delete_items):
    r = dict(d)
    for key in delete_items: 
        del r[key]
    return r

normal_country_dict = removekey(country_dict, special)
print(len(normal_country_dict.values()))
normal_countr = list(normal_country_dict.keys())

183


In [34]:
def portion_data(country):
    data = normal_country_dict[country]
    tier = ["TIER 1", "TIER 2", "TIER 3"]
    for i in tier:
        length = len(data[0]["INTRO"].split(i))       
        if "TIER 2 WATCH LIST" in data[0]["INTRO"]:
            i = "TIER 2 WATCH LIST"
            append_data(country,i,data)
        elif length == 2 and "TIER 2 WATCH LIST" not in data[0]["INTRO"]: 
            append_data(country,i,data)            
    return data,country_dict


In [35]:
def append_data(country,i,data):
    data[0]["INTRO"] = data[0]["INTRO"].split(i)[1]
    data.append({"TIER":i})
    intro = data[0]["INTRO"].split(f"RECOMMENDATIONS FOR {country}")
    data[0]["INTRO"] = intro[0]
    recommend = intro[1].split("PROSECUTION")
    data.append({"RECOMMENDATIONS":recommend[0]})
    prosecute = recommend[1].split("PROTECTION")
    data.append({"PROSECUTIONS":prosecute[0]})
    protect = prosecute[1].split("PREVENTION")
    data.append({"PROTECTION":protect[0]})
    prevent = protect[1].split("TRAFFICKING PROFILE")
    data.append({"PREVENTION":prevent[0]})
    data.append({"TRAFFICKING PROFILE":prevent[1]})
    return data

In [37]:
new_normal_dict = {}
for i in normal_countr:
    try:
        new_normal_dict[i] = portion_data(i)
    except:
        pass

In [38]:
x = countries[:183]  #183
new_normal_dict = [{i.upper():portion_data(i.upper())[0]} for i in x]


In [56]:
trafficking_profile = [{a:list(new_normal_dict[normal_countr.index(a)][a][-1].values())[0]} for a,b in zip(normal_countr, new_normal_dict)]
print(trafficking_profile)

[{'AFGHANISTAN': 'As reported over the past five years, Afghanistan is a source, transit, and destination country for men, women, and children subjected to forced labor and sex trafficking. Internal trafficking is more prevalent than transnational trafficking. Men, women, and children are exploited in bonded labor in Afghanistan, where an initial debt assumed by a worker as part of the terms of employment is exploited, ultimately entrapping other family members, sometimes for multiple generations. There are entire Afghan families trapped in debt bondage in the brick-making industry in eastern Afghanistan. Most Afghan trafficking victims are children exploited in carpet making and brick factories, domestic servitude, commercial sex, begging, poppy cultivation, salt mining, transnational drug smuggling, and assistant truck driving within Afghanistan. Some Afghan families knowingly sell their children into sex trafficking, including for bacha bazi, including some government officials and 

In [57]:
af = trafficking_profile[0]
af['AFGHANISTAN'].split(".")

['As reported over the past five years, Afghanistan is a source, transit, and destination country for men, women, and children subjected to forced labor and sex trafficking',
 ' Internal trafficking is more prevalent than transnational trafficking',
 ' Men, women, and children are exploited in bonded labor in Afghanistan, where an initial debt assumed by a worker as part of the terms of employment is exploited, ultimately entrapping other family members, sometimes for multiple generations',
 ' There are entire Afghan families trapped in debt bondage in the brick-making industry in eastern Afghanistan',
 ' Most Afghan trafficking victims are children exploited in carpet making and brick factories, domestic servitude, commercial sex, begging, poppy cultivation, salt mining, transnational drug smuggling, and assistant truck driving within Afghanistan',
 ' Some Afghan families knowingly sell their children into sex trafficking, including for bacha bazi, including some government officials 

In [66]:
nig = trafficking_profile[125]
nig['NIGERIA'].split(".")

['As reported over the past five years, Nigeria is a source, transit, and destination country for women and children subjected to forced labor and sex trafficking, and a source country for men subjected to forced labor',
 " Nigerian trafficking victims are recruited from rural areasŠespecially the country's southern regionsŠand, to a lesser extent, urban areas",
 ' Women and girls are victims of domestic servitude and sex trafficking, and boys are victims of forced and bonded labor in street vending, domestic service, mining, stone quarrying, agriculture, textile manufacturing, and begging',
 ' The government estimates as many as 9',
 '5 million young boys studying in Quranic schools, commonly known as Almajiri, are subjected to forced begging',
 ' Traffickers operate ﬁbaby factoriesﬂŠoften disguised as orphanages, maternity homes, or religious centersŠwhere traffickers hold women against their will, rape them, and force them to carry and deliver a child',
 ' The traffickers sell the c

In [285]:
country_short = [i[:6] for i in countries]
gender_class = ["men", "women", "boy", "girl", "child"]
trafficking_type = ["forced", "labor", "sex", "bondage"]

In [1217]:
combinations = [[f,s,t] for f in country_short for s in gender_class for t in trafficking_type]
combinations

[['Afghan', 'men', 'forced'],
 ['Afghan', 'men', 'labor'],
 ['Afghan', 'men', 'sex'],
 ['Afghan', 'men', 'bondage'],
 ['Afghan', 'women', 'forced'],
 ['Afghan', 'women', 'labor'],
 ['Afghan', 'women', 'sex'],
 ['Afghan', 'women', 'bondage'],
 ['Afghan', 'boy', 'forced'],
 ['Afghan', 'boy', 'labor'],
 ['Afghan', 'boy', 'sex'],
 ['Afghan', 'boy', 'bondage'],
 ['Afghan', 'girl', 'forced'],
 ['Afghan', 'girl', 'labor'],
 ['Afghan', 'girl', 'sex'],
 ['Afghan', 'girl', 'bondage'],
 ['Afghan', 'child', 'forced'],
 ['Afghan', 'child', 'labor'],
 ['Afghan', 'child', 'sex'],
 ['Afghan', 'child', 'bondage'],
 ['Albani', 'men', 'forced'],
 ['Albani', 'men', 'labor'],
 ['Albani', 'men', 'sex'],
 ['Albani', 'men', 'bondage'],
 ['Albani', 'women', 'forced'],
 ['Albani', 'women', 'labor'],
 ['Albani', 'women', 'sex'],
 ['Albani', 'women', 'bondage'],
 ['Albani', 'boy', 'forced'],
 ['Albani', 'boy', 'labor'],
 ['Albani', 'boy', 'sex'],
 ['Albani', 'boy', 'bondage'],
 ['Albani', 'girl', 'forced'],
 ['Al

In [ ]:
b = [i for i in text for j in combinations if all(k in i for k in j)]


In [2]:
from bs4 import BeautifulSoup
import json
import urllib.request
url = 'tolu1.ipynb'
# response = urllib.request.urlopen(url)
#  for local html file
response = open("tolu1.ipynb.html")


In [3]:
text = response.read()

soup = BeautifulSoup(text, 'lxml')
# see some of the html
print(soup.div)
dictionary = {'nbformat': 4, 'nbformat_minor': 1, 'cells': [], 'metadata': {}}
for d in soup.findAll("div"):
    if 'class' in d.attrs.keys():
        for clas in d.attrs["class"]:
            if clas in ["text_cell_render", "input_area"]:
                # code cell
                if clas == "input_area":
                    cell = {}
                    cell['metadata'] = {}
                    cell['outputs'] = []
                    cell['source'] = [d.get_text()]
                    cell['execution_count'] = None
                    cell['cell_type'] = 'code'
                    dictionary['cells'].append(cell)

                else:
                    cell = {}
                    cell['metadata'] = {}

                    cell['source'] = [d.decode_contents()]
                    cell['cell_type'] = 'markdown'
                    dictionary['cells'].append(cell)
open('notebook.ipynb', 'w').write(json.dumps(dictionary))

None


65